In [1]:
import os
import pathlib
import tensorflow as tf

from tensorflow import keras as keras
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.optimizers import Adam
from keras.applications.mobilenet_v2 import MobileNetV2

In [ ]:
NUM_CLASSES = 2

TRAIN_DATA_PATH = '/home/mist/mask_datasets/train/'
VALID_DATA_PATH = '/home/mist/mask_datasets/valid/'

BATCH_SIZE = 32
TRAIN_EPOCHS = 15
IMG_HEIGHT = 224
IMG_WIDTH = 224

train_datagen = ImageDataGenerator(
    rotation_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DATA_PATH,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True)  # Set as training dataset

validation_generator = valid_datagen.flow_from_directory(
    VALID_DATA_PATH,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True)  # Set as validation dataset

print(train_generator.class_indices)
print(validation_generator.class_indices)

In [ ]:
import matplotlib.pyplot as plt

# Check for valid images
g = os.walk(TRAIN_DATA_PATH)
for path,dir_list,file_list in g:  
    for file_name in file_list:
        try:
            plt.imread(path + '/'+ file_name)
        except:
            print(path + "/" + file_name)
print("Check for train images")

g = os.walk(VALID_DATA_PATH)
for path,dir_list,file_list in g:  
    for file_name in file_list:
        try:
            plt.imread(path + '/'+ file_name)
        except:
            print(path + "/" + file_name)

print("Check for valid images")

In [ ]:
base_model = MobileNetV2(weights='imagenet',
                         include_top=False,
                         input_shape=[IMG_HEIGHT, IMG_WIDTH, 3],
                         classes=NUM_CLASSES)

avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
dense = keras.layers.Dense(1024, activation = 'relu')(avg)
dense = keras.layers.Dense(1024, activation = 'relu')(dense)
dense = keras.layers.Dense(512, activation = 'relu')(dense)
preds = keras.layers.Dense(NUM_CLASSES, activation = 'sigmoid')(dense)
model = keras.Model(inputs = base_model.input, outputs = preds)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5, 
    patience=3, 
    verbose=1)

for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True
    
model.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(lr=1e-4),
              metrics = ['binary_accuracy'])

model.fit(
    train_generator,
    epochs=TRAIN_EPOCHS,
    steps_per_epoch=train_generator.n//train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.n//validation_generator.batch_size,
    callbacks=[reduce_lr])

model.save('mask_recongnition_model')